In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.tseries.offsets import MonthEnd
import warnings
import os

# warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Access data directory
os.chdir("../data")
print("Current directory (should be data):")
os.getcwd()

Current directory (should be data):


'/home/blshen/ml-costs-thesis/data'

In [8]:
# LOAD LINK TABLE

linker = pd.read_csv('linker.csv')

# Fix dates
linker['LINKDT'] = pd.to_datetime(linker['LINKDT'], errors='coerce')
linker['LINKENDDT'] = linker['LINKENDDT'].replace(['E', 'e'], pd.NaT)
linker['LINKENDDT'] = pd.to_datetime(linker['LINKENDDT'], errors='coerce')
linker['LINKENDDT'] = linker['LINKENDDT'].fillna(pd.Timestamp('2024-12-31'))

# Validation Checks
print(f"   Loaded {len(linker):,} linking records")
print(f"   Unique GVKEYs: {linker['GVKEY'].nunique():,}")
print(f"   Unique PERMNOs: {linker['LPERMNO'].nunique():,}")

# Rename for consistency
linker = linker.rename(columns={
    'GVKEY': 'gvkey',
    'LPERMNO': 'PERMNO'
})

   Loaded 1,650 linking records
   Unique GVKEYs: 1,264
   Unique PERMNOs: 1,274


In [9]:
# LOAD COMPUSTAT (Fundamental Data)

# Quarterly
quarterly = pd.read_csv('quarterly_fundamentals.csv')
quarterly['datadate'] = pd.to_datetime(quarterly['datadate'])
quarterly['gvkey'] = quarterly['gvkey'].astype(str).str.zfill(6)
print(f"   {len(quarterly):,} quarterly observations")

# Yearly
yearly = pd.read_csv('yearly_fundamentals.csv')
yearly['datadate'] = pd.to_datetime(yearly['datadate'])
yearly['gvkey'] = yearly['gvkey'].astype(str).str.zfill(6)
print(f"   {len(yearly):,} annual observations")

# Convert keys to strings
quarterly['gvkey'] = quarterly['gvkey'].astype(str)
yearly['gvkey'] = yearly['gvkey'].astype(str)
linker['gvkey'] = linker['gvkey'].astype(str)
linker['PERMNO'] = pd.to_numeric(linker['PERMNO'], errors='coerce')
linker = linker.dropna(subset=['PERMNO'])
linker['PERMNO'] = linker['PERMNO'].astype(int).astype(str).str.zfill(5)

print("\n---\n")

# Print data columns
print("Quarterly fundamentals (head):")
print(quarterly.head(5))
print("\nYearly fundamentals (head):")
print(yearly.head(5))

   109,015 quarterly observations
   27,125 annual observations

---

Quarterly fundamentals (head):
  costat curcdq datafmt indfmt consol   gvkey   datadate  fyr datacqtr  \
0      I    USD     STD   INDL      C  001013 1993-01-31   10   1992Q4   
1      I    USD     STD   INDL      C  001013 1993-04-30   10   1993Q1   
2      I    USD     STD   INDL      C  001013 1993-07-31   10   1993Q2   
3      I    USD     STD   INDL      C  001013 1993-10-31   10   1993Q3   
4      I    USD     STD   INDL      C  001013 1994-01-31   10   1993Q4   

          rdq      atq     ceqq   cogsq    dpq     ibq     niq  ppentq  \
0  1993-02-22  238.483  190.883  33.526  4.984   5.260   5.260  57.347   
1  1993-05-25  244.219  198.516  38.600  5.108   7.064   7.064  58.949   
2  1993-08-24  255.495  208.328  39.556  5.152   9.181   9.181  61.974   
3  1993-12-16  280.054  220.394  46.303  5.343  10.131  10.131  62.876   
4  1994-02-22  280.447  226.487  40.412  5.619   6.695   5.245  61.606   

     sale

In [4]:
# DATA CLEANING (For Fundamentals Data)

# Filter data
quarterly = quarterly[
    (quarterly['datafmt'] == 'STD') &
    (quarterly['indfmt'] == 'INDL') &
    (quarterly['consol'] == 'C') &
    (quarterly['curcdq'] == 'USD')
]
yearly = yearly[
    (yearly['datafmt'] == 'STD') &
    (yearly['indfmt'] == 'INDL') &
    (yearly['consol'] == 'C') &
    (yearly['curcd'] == 'USD')
]

# Drop redundant columns
cols_to_drop = [
    'costat',   # company status (active/inactive)
    'datafmt',  # format type (always 'STD' now)
    'indfmt',   # industry format (always 'INDL')
    'consol',   # consolidation code (always 'C')
    'curcd', 'curcdq',  # currency codes (always 'USD')
    'popsrc',   # population source, if present
    'tic',      # ticker (unstable over time)
    'cusip'     # redundant when linking via PERMNO
]
quarterly = quarterly.drop(columns=[c for c in cols_to_drop if c in quarterly.columns])
yearly = yearly.drop(columns=[c for c in cols_to_drop if c in yearly.columns])

# Print data columns
print("Quarterly fundamentals (head):")
print(quarterly.head(5))
print("\nYearly fundamentals (head):")
print(yearly.head(5))

Quarterly fundamentals (head):
    gvkey   datadate  fyr datacqtr         rdq      atq     ceqq   cogsq  \
0  001013 1993-01-31   10   1992Q4  1993-02-22  238.483  190.883  33.526   
1  001013 1993-04-30   10   1993Q1  1993-05-25  244.219  198.516  38.600   
2  001013 1993-07-31   10   1993Q2  1993-08-24  255.495  208.328  39.556   
3  001013 1993-10-31   10   1993Q3  1993-12-16  280.054  220.394  46.303   
4  001013 1994-01-31   10   1993Q4  1994-02-22  280.447  226.487  40.412   

     dpq     ibq     niq  ppentq    saleq   txtq    xrdq   xsgaq  oancfy  
0  4.984   5.260   5.260  57.347   78.648  3.090   9.537  30.965   9.565  
1  5.108   7.064   7.064  58.949   88.999  4.148  10.282  33.318   9.586  
2  5.152   9.181   9.181  61.974   93.346  5.164  10.080  33.515  21.776  
3  5.343  10.131  10.131  62.876  105.125  5.699  11.089  36.501  29.448  
4  5.619   6.695   5.245  61.606   91.176  3.931  11.003  34.882  20.251  

Yearly fundamentals (head):
    gvkey   datadate  fyr  fyear 

In [5]:
# PERFORM LINK (Compustat -> CRSP) VIA PERMNO

def add_permno(df, linker, source_name, drop_gvkey):
    """
    Add PERMNO to Compustat data using time-aware linking
    """
    
    # --- Standardize dtypes ---
    df = df.copy()
    df['gvkey'] = df['gvkey'].astype(str).str.zfill(6)
    linker = linker.copy()
    linker['gvkey'] = linker['gvkey'].astype(str).str.zfill(6)
    
    # --- Record baseline counts ---
    total_obs = len(df)
    total_gvkeys = df['gvkey'].nunique()
    
    # --- Merge ---
    df = df.merge(
        linker[['gvkey', 'PERMNO', 'LINKDT', 'LINKENDDT']],
        on='gvkey',
        how='left'
    )
    
    # --- Filter by link validity ---
    df = df[
        (df['datadate'] >= df['LINKDT']) &
        (df['datadate'] <= df['LINKENDDT'])
    ].copy()
    
    # --- Drop linking columns ---
    df.drop(columns=['LINKDT', 'LINKENDDT'], inplace=True)
    
    # --- Drop gvkey after rekeying ---
    if drop_gvkey:
        df.drop(columns=['gvkey'], inplace=True, errors='ignore')
    
    # --- Validation summary ---
    linked_obs = len(df)
    linked_permnos = df['PERMNO'].notna().sum()
    linked_gvkeys = df['PERMNO'].notna().sum()
    
    print(f"   {source_name}: {linked_obs:,} / {total_obs:,} observations linked ({linked_obs / total_obs:.1%})")
    
    return df

# Add PERMNO to fundamentals
quarterly = add_permno(quarterly, linker, "Quarterly", True)
yearly = add_permno(yearly, linker, "Yearly", True)

# Clear memory
del linker

   Quarterly: 101,665 / 109,015 observations linked (93.3%)
   Yearly: 25,366 / 27,125 observations linked (93.5%)


In [6]:
# LOAD MARKET DATA (CRSP)

# Lean import
keep_cols = [
    'PERMNO', 'Ticker', 'MthCalDt', 'MthPrc', 
    'MthCap', 'MthRet', 'MthVol', 'ShrOut'
]
market_data = pd.read_csv('market_data.csv', usecols=keep_cols)
market_data['MthCalDt'] = pd.to_datetime(market_data['MthCalDt'])
market_data['MthCalDt'] = market_data['MthCalDt'] + MonthEnd(0)
market_data['PERMNO'] = market_data['PERMNO'].astype(str).str.zfill(5)

market_data = market_data.sort_values(['PERMNO', 'MthCalDt'])

# Validation
print(f"   Loaded {len(market_data):,} monthly stock observations")
print(f"   Date range: {market_data['MthCalDt'].min()} to {market_data['MthCalDt'].max()}")
print(f"   Unique stocks: {market_data['PERMNO'].nunique():,}")

# Print data columns
print("Market data (head):")
print(market_data.head(5))

   Loaded 303,111 monthly stock observations
   Date range: 1993-01-31 00:00:00 to 2024-12-31 00:00:00
   Unique stocks: 1,275
Market data (head):
  PERMNO Ticker   MthCalDt  MthPrc      MthCap    MthRet      MthVol  ShrOut
0  10078   SUNW 1993-01-31  39.000  4090359.00  0.159851  42128666.0  104881
1  10078   SUNW 1993-02-28  35.125  3683945.13 -0.099359  28023855.0  104881
2  10078   SUNW 1993-03-31  30.000  3173400.00 -0.145907  49896054.0  105780
3  10078   SUNW 1993-04-30  27.000  2856060.00 -0.100000  39674294.0  105780
4  10078   SUNW 1993-05-31  30.000  3173400.00  0.111111  29577640.0  105780


In [7]:
# CONSTRUCT MONTHLY PRICE-BASED FEATURES

def construct_price_features(df):
    """
    Construct key predictive features from price/return data
    Following Gu, Kelly, and Xiu (2020)
    """
    
    features = df.copy()
    
    # 1. SIZE: Log market cap
    features['mvel1'] = np.log(features['MthCap'])
    
    # 2. MOMENTUM FEATURES
    # mom1m: 1-month return (t-1)
    features['mom1m'] = features.groupby('PERMNO')['MthRet'].shift(1)
    
    # mom6m: 6-month momentum (t-6 to t-1, skip most recent)
    features['mom6m'] = features.groupby('PERMNO')['MthRet'].transform(
        lambda x: x.shift(1).rolling(6, min_periods=4).apply(lambda r: (1 + r).prod() - 1)
    )
    
    # mom12m: 12-month momentum (t-12 to t-2, skip most recent)
    features['mom12m'] = features.groupby('PERMNO')['MthRet'].transform(
        lambda x: x.shift(2).rolling(11, min_periods=8).apply(lambda r: (1 + r).prod() - 1)
    )
    
    # mom36m: 36-month momentum (long-term reversal, t-36 to t-13)
    features['mom36m'] = features.groupby('PERMNO')['MthRet'].transform(
        lambda x: x.shift(13).rolling(24, min_periods=18).apply(lambda r: (1 + r).prod() - 1)
    )
    
    # chmom: Change in 6-month momentum
    features['chmom'] = features.groupby('PERMNO')['mom6m'].diff(6)

    print(f"   Constructed momentum features: mom1m, mom6m, mom12m, mom36m, chmom")
    
    # 3. VOLATILITY FEATURES
    # retvol: Return volatility (12-month rolling)
    features['retvol'] = features.groupby('PERMNO')['MthRet'].transform(
        lambda x: x.rolling(12, min_periods=6).std()
    )

    print(f"   Constructed volatility features: retvol")
    
    # 4. LIQUIDITY FEATURES
    # dolvol: Log dollar volume
    dvol = features['MthVol'] * features['MthPrc']
    dvol = dvol.where(dvol > 0, np.nan)
    features['dolvol'] = np.log(dvol)
    
    # turn: Share turnover
    features['turn'] = features['MthVol'] / features['ShrOut']
    features['turn'] = features['turn'].replace([np.inf, -np.inf], np.nan)
    
    # std_turn: Volatility of turnover
    features['std_turn'] = features.groupby('PERMNO')['turn'].transform(
        lambda x: x.rolling(12, min_periods=6).std()
    )

    print(f"   Constructed liquidity features: mvel1, dolvol, turn, std_turn")
    
    return features

market_features = construct_price_features(market_data)
del market_data

   Constructed momentum features: mom1m, mom6m, mom12m, mom36m, chmom
   Constructed volatility features: retvol
   Constructed liquidity features: mvel1, dolvol, turn, std_turn


In [9]:
def process_daily_data(daily_csv_path, chunk_size=1_000_000):
    """
    Processes a large daily data CSV in chunks to calculate
    monthly aggregates for bid-ask spread and max daily return.

    This function correctly handles chunk boundaries by delaying
    aggregation until all data is loaded.

    Args:
        daily_csv_path (str): The path to the 'daily.csv' file.
        chunk_size (int): The number of rows to read per chunk.

    Returns:
        pd.DataFrame: A DataFrame with PERMNO, MthCalDt,
                      baspread (monthly avg), and maxret (monthly max).
    """
    
    daily_chunks = []

    dtype_map = {
        'PERMNO': 'Int64',
        'DlyCalDt': 'string',
        'DlyAsk': 'float64',
        'DlyBid': 'float64',
        'DlyRet': 'float64'
    }
    
    # We must iterate with an index to get the first chunk for column validation
    for i, chunk in enumerate(pd.read_csv(
        daily_csv_path, 
        chunksize=chunk_size, 
        dtype=dtype_map,
        low_memory=False)):
        
        if i == 0:
            # Validate required columns (PERMNO is essential)
            required_cols = ['PERMNO', 'DlyCalDt', 'DlyAsk', 'DlyBid', 'DlyRet']
            if not all(col in chunk.columns for col in required_cols):
                print(f"   Error: missing {required_cols}")
                print(f"   Found: {list(chunk.columns)}")
                return pd.DataFrame()
        
        # --- 1. Process Daily Data (Inside Chunk) ---
        
        # Convert date
        chunk['DlyCalDt'] = pd.to_datetime(chunk['DlyCalDt'], errors='coerce')
        
        # Calculate daily spread
        chunk['daily_spread'] = np.where(
            (chunk['DlyAsk'] + chunk['DlyBid']) > 0,
            (chunk['DlyAsk'] - chunk['DlyBid']) / ((chunk['DlyAsk'] + chunk['DlyBid']) / 2),
            np.nan
        )
        chunk = chunk[['PERMNO', 'DlyCalDt', 'DlyRet', 'daily_spread']]
        
        daily_chunks.append(chunk)

    print(f"   All chunks loaded")
    
    # --- 2. Aggregate (After Concatenation) ---
    
    daily_df = pd.concat(daily_chunks, ignore_index=True)
    daily_df = daily_df.dropna(subset=['PERMNO', 'DlyCalDt'])
    
    print(f"   Daily data concatenated. Total daily observations: {len(daily_df):,}")
    
    # Create the monthly grouping key
    daily_df['year_month'] = daily_df['DlyCalDt'].dt.to_period('M')
    
    print("   Aggregating daily data to monthly features...")
    
    # Now, perform a single, correct aggregation
    monthly_agg = daily_df.groupby(['PERMNO', 'year_month']).agg(
        # Calculate mean bid-ask spread for the month
        baspread=('daily_spread', 'mean'), 
        
        # Find the maximum daily return for the month
        maxret=('DlyRet', 'max') # This is the new, accurate maxret
    ).reset_index()

    print(f"   Aggregation complete. {len(monthly_agg):,} stock-month observations created.")
    
    # --- 3. Final Formatting ---
    
    # Convert 'year_month' to a month-end timestamp
    monthly_agg['MthCalDt'] = monthly_agg['year_month'].dt.to_timestamp('M') + pd.offsets.MonthEnd(0)
    
    # Drop the temporary key
    monthly_agg = monthly_agg.drop(columns='year_month')
    
    return monthly_agg

daily_features_df = process_daily_data('daily.csv')
daily_features_df['PERMNO'] = daily_features_df['PERMNO'].astype(str)
daily_features_df['MthCalDt'] = pd.to_datetime(daily_features_df['MthCalDt'])

# Merge the daily-based features
market_features = market_features.merge(
    daily_features_df,
    on=['PERMNO', 'MthCalDt'],
    how='left'
)

del daily_features_df

print("\n   Merged daily features and updated 'baspread' and 'maxret' columns.")
print("\n--- Updated Market Features (Head) ---")
print(market_features.head())

   All chunks loaded
   Daily data concatenated. Total daily observations: 6,327,407
   Aggregating daily data to monthly features...
   Aggregation complete. 302,063 stock-month observations created.

   Merged daily features and updated 'baspread' and 'maxret' columns.

--- Updated Market Features (Head) ---
  PERMNO Ticker   MthCalDt  MthPrc      MthCap    MthRet      MthVol  ShrOut  \
0  10078   SUNW 1993-01-31  39.000  4090359.00  0.159851  42128666.0  104881   
1  10078   SUNW 1993-02-28  35.125  3683945.13 -0.099359  28023855.0  104881   
2  10078   SUNW 1993-03-31  30.000  3173400.00 -0.145907  49896054.0  105780   
3  10078   SUNW 1993-04-30  27.000  2856060.00 -0.100000  39674294.0  105780   
4  10078   SUNW 1993-05-31  30.000  3173400.00  0.111111  29577640.0  105780   

       mvel1     mom1m     mom6m  mom12m  mom36m  chmom  retvol     dolvol  \
0  15.224143       NaN       NaN     NaN     NaN    NaN     NaN  21.219801   
1  15.119495  0.159851       NaN     NaN     NaN   

In [10]:
# CONSTRUCT FUNDAMENTALS-BASED FEATURES

def construct_fundamental_features(quarterly, yearly):
    """
    Construct fundamental features from Compustat data.
    This version correctly implements GKX features.
    """
    
    # 1. YEARLY FEATURES
    yearly = yearly.sort_values(['PERMNO', 'datadate'])
    
    # 'be': Book Equity (uses 'ceq' - Common Equity)
    # This is a point-in-time value for the fiscal year-end.
    yearly['be'] = yearly['ceq']
    yearly = yearly.drop('ceq', axis=1)

    # 'agr': Asset Growth (year-over-year % change in 'at' - Total Assets)
    # This is (at[t] - at[t-1]) / at[t-1], known at time t.
    yearly['agr'] = yearly.groupby('PERMNO')['at'].pct_change(fill_method=None)
    
    # 1. Calculate Operating Profit
    # (FillNa(0) assumes missing cogs/xsga are 0, which is standard)
    yearly['op'] = yearly['sale'].fillna(0) - yearly['cogs'].fillna(0) - yearly['xsga'].fillna(0)
    
    # 2. Calculate Ratio (using np.where to avoid divide-by-zero)
    yearly['operprof'] = np.where(
        yearly['be'] > 0,  # Only calculate if Book Equity is positive
        yearly['op'] / yearly['be'],
        np.nan
    )
    
    # 'ep_numerator': Earnings (Numerator for E/P ratio)
    # Uses 'ib' (Income Before Extraordinary Items - Yearly)
    yearly['ep_numerator'] = yearly['ib']
    
    # 'rd_numerator': R&D (Numerator for R&D/MVE ratio)
    # Uses 'xrd' (R&D Expense). Fill missing with 0.
    yearly['rd_numerator'] = yearly['xrd'].fillna(0)

    print("   Constructed yearly features: be, agr, op, operprof, ep_numerator, rd_numerator")
    
    # 2. QUARTERLY FEATURES
    quarterly = quarterly.sort_values(['PERMNO', 'rdq'])
    
    # 'earnings_ttm': Trailing 12-Month Earnings
    # Sum of the last 4 quarters of 'ibq' (Income - Quarterly)
    quarterly['earnings_ttm'] = quarterly.groupby('PERMNO')['ibq'].transform(
        lambda x: x.rolling(4, min_periods=4).sum()
    )
    
    print("   Constructed quarterly feature: earnings_ttm")
    
    return yearly, quarterly

yearly_features, quarterly_features = construct_fundamental_features(
    quarterly, yearly
)

del quarterly
del yearly

   Constructed yearly features: be, agr, op, operprof, ep_numerator, rd_numerator
   Constructed quarterly feature: earnings_ttm


In [11]:
## CREATE POINT-IN-TIME FUNDAMENTALS

def create_point_in_time_fundamentals(market_df, fundamental_df):
    """
    Create point-in-time fundamental features for each stock-month.
    Handles reporting lag to avoid look-ahead bias.
    
    Args:
        market_df: Monthly market data with PERMNO and MthCalDt
        fundamental_df: Fundamental data with PERMNO already linked
    
    Returns:
        DataFrame with point-in-time fundamental features merged
    """
    
    # Prepare fundamental data with reporting lag
    fund = fundamental_df.copy()
    
    # Add 4-month reporting lag to avoid look-ahead bias
    # Fundamentals reported on datadate are available 4 months later
    fund['available_date'] = fund['datadate'] + pd.DateOffset(months=4)
    
    # Select columns needed for matching
    fund_columns = ['PERMNO', 'available_date', 'be', 'agr', 'operprof', 
                    'at', 'ep_numerator', 'rd_numerator']
    fund = fund[fund_columns].copy()
    
    # Remove any rows with missing critical data
    fund = fund.dropna(subset=['PERMNO', 'available_date'])
    
    # Remove duplicates - keep most recent if multiple records on same available_date
    fund = fund.sort_values(['PERMNO', 'available_date', 'be'])
    fund = fund.drop_duplicates(subset=['PERMNO', 'available_date'], keep='last')
    
    print(f"   Prepared {len(fund):,} fundamental records with reporting lag")
    
    # Sort both dataframes for merge_asof
    market = market_df.sort_values(['PERMNO', 'MthCalDt']).reset_index(drop=True)
    fund = fund.sort_values(['PERMNO', 'available_date']).reset_index(drop=True)
    
    # Perform point-in-time merge using merge_asof by PERMNO
    merged_list = []
    
    for permno in market['PERMNO'].unique():
        # Get data for this stock
        market_stock = market[market['PERMNO'] == permno].copy()
        fund_stock = fund[fund['PERMNO'] == permno].copy()
        
        if len(fund_stock) == 0:
            # No fundamental data for this stock - keep market data as-is
            merged_list.append(market_stock)
            continue
        
        # Point-in-time merge: for each MthCalDt, use most recent available fundamental
        # direction='backward' means we look back for the latest available data
        merged = pd.merge_asof(
            market_stock,
            fund_stock.drop(columns=['PERMNO']),  # Drop PERMNO since it's in left df
            left_on='MthCalDt',
            right_on='available_date',
            direction='backward',
            allow_exact_matches=True
        )
        
        merged_list.append(merged)
    
    # Combine all stocks
    result = pd.concat(merged_list, ignore_index=True)
    
    print(f"   Completed point-in-time matching for {result['PERMNO'].nunique():,} stocks")
    
    # Compustat fundamentals are in millions, CRSP MthCap is in thousands
    result['MthCap_millions'] = result['MthCap'] / 1000
    
    # Calculate ratio features using merged data
    # Book-to-market ratio
    result['bm'] = result['be'] / result['MthCap_millions']
    result['bm'] = result['bm'].replace([np.inf, -np.inf], np.nan)
    
    # Earnings-to-price ratio (if ep_numerator exists)
    if 'ep_numerator' in result.columns:
        result['ep'] = result['ep_numerator'] / result['MthCap_millions']
        result['ep'] = result['ep'].replace([np.inf, -np.inf], np.nan)
    
    # R&D-to-market-cap ratio (if rd_numerator exists)
    if 'rd_numerator' in result.columns:
        result['rd_mve'] = result['rd_numerator'] / result['MthCap_millions']
        result['rd_mve'] = result['rd_mve'].replace([np.inf, -np.inf], np.nan)
    
    # Drop intermediate columns
    result = result.drop(columns=['available_date', 'MthCap_millions'], errors='ignore')
    
    return result

# Apply point-in-time matching to yearly fundamentals
all_features = create_point_in_time_fundamentals(market_features, yearly_features)
print(f"   Final dataset: {len(all_features):,} stock-month observations")

   Prepared 24,991 fundamental records with reporting lag
   Completed point-in-time matching for 1,275 stocks
   Final dataset: 303,111 stock-month observations


In [12]:
# Clean up dataframes no longer needed
'''
del quarterly_features
del yearly_features
del market_features
'''

# Remove unnecessary columns
columns_to_drop = ['MthPrc', 'MthCap', 'MthVol', 'ShrOut', 'be', 'at', 'ep_numerator', 'rd_numerator', 'mom36m']
all_features = all_features.drop(columns=columns_to_drop, errors='ignore')

print("\n--- All Features (Head) ---")
print(all_features.head())


--- All Features (Head) ---
  PERMNO Ticker   MthCalDt    MthRet      mvel1     mom1m     mom6m  mom12m  \
0  10078   SUNW 1993-01-31  0.159851  15.224143       NaN       NaN     NaN   
1  10078   SUNW 1993-02-28 -0.099359  15.119495  0.159851       NaN     NaN   
2  10078   SUNW 1993-03-31 -0.145907  14.970314 -0.099359       NaN     NaN   
3  10078   SUNW 1993-04-30 -0.100000  14.864954 -0.145907       NaN     NaN   
4  10078   SUNW 1993-05-31  0.111111  14.970314 -0.100000 -0.197026     NaN   

   chmom  retvol     dolvol        turn  std_turn  baspread    maxret  agr  \
0    NaN     NaN  21.219801  401.680629       NaN  0.005818  0.081784  NaN   
1    NaN     NaN  20.707480  267.196680       NaN  0.005723  0.036101  NaN   
2    NaN     NaN  21.126650  471.696483       NaN  0.005474  0.048673  NaN   
3    NaN     NaN  20.792051  375.064228       NaN  0.007721  0.034632  NaN   
4    NaN     NaN  20.603727  279.614672       NaN  0.007193  0.074236  NaN   

   operprof  bm  ep  rd_mve

In [13]:
# CREATE S&P 500 MEMBERSHIP INDICATOR

# LOAD S&P 500 MEMBERSHIP DATA
membership = pd.read_csv('membership.csv')
membership['start'] = pd.to_datetime(membership['start'])
membership['ending'] = pd.to_datetime(membership['ending'])
membership = membership.rename(columns={'permno': 'PERMNO'})
membership['PERMNO'] = pd.to_numeric(membership['PERMNO'], errors='coerce')
membership = membership.dropna(subset=['PERMNO'])
membership['PERMNO'] = membership['PERMNO'].astype(int).astype(str).str.zfill(5)

print(f"   Loaded {len(membership):,} membership records")
print(f"   Unique PERMNOs: {membership['PERMNO'].nunique():,}")

# Create the indicator
def create_sp500_indicator(market_df, membership_df):
    """
    Create point-in-time indicator for S&P 500 membership.
    Expands membership periods into monthly indicators to avoid look-ahead bias.
    
    Args:
        market_df: Market data with PERMNO and MthCalDt
        membership_df: S&P membership with PERMNO, start, ending dates
    
    Returns:
        DataFrame with in_sp500 binary indicator added
    """
    
    membership_expanded = []
    
    for idx, row in membership_df.iterrows():
        # Create date range for this membership period (month-end dates)
        dates = pd.date_range(row['start'], row['ending'], freq='ME')
        
        for date in dates:
            membership_expanded.append({
                'PERMNO': row['PERMNO'],
                'MthCalDt': date,
                'in_sp500': 1
            })
    
    sp500_indicator = pd.DataFrame(membership_expanded)
    
    # Merge with market data (left join preserves all market observations)
    result = market_df.merge(
        sp500_indicator,
        on=['PERMNO', 'MthCalDt'],
        how='left'
    )
    
    # Fill missing with 0 (not in S&P 500)
    result['in_sp500'] = result['in_sp500'].fillna(0).astype(int)
    
    return result

# Apply S&P 500 indicator to all_features
all_features = create_sp500_indicator(all_features, membership)

print(f"   Added S&P 500 membership indicator")
print(f"   Non-S&P 500 stock-months: {(all_features['in_sp500']==0).sum():,}")
print(f"   S&P 500 coverage: {all_features['in_sp500'].mean()*100:.1f}% of stock-months")

   Loaded 1,308 membership records
   Unique PERMNOs: 1,275
   Added S&P 500 membership indicator
   Non-S&P 500 stock-months: 109,976
   S&P 500 coverage: 63.7% of stock-months


In [14]:
# DATA VALIDATION
print("\n" + "="*80)
print("DATA VALIDATION")
print("="*80)

# Define feature set
feature_list = [
    # Momentum features
    'mom1m', 'mom6m', 'mom12m', 'mom36m', 'chmom',
    # Size & liquidity
    'mvel1', 'dolvol', 'turn', 'std_turn',
    # Volatility
    'retvol', 'maxret',
    # Spread
    'baspread',
    # Fundamentals
    'bm', 'agr', 'operprof',
]

# 1. CHECK DATE ALIGNMENT
print("\n1. DATE ALIGNMENT")
print("-" * 40)
print("Month-end day distribution:")
print(all_features['MthCalDt'].dt.day.value_counts().head())
assert all_features['MthCalDt'].dt.is_month_end.all(), "❌ Not all dates are month-end!"
print("✓ All dates are proper month-ends")

# 2. CHECK PERMNO FORMAT
print("\n2. PERMNO FORMAT")
print("-" * 40)
print(f"Sample PERMNOs: {all_features['PERMNO'].head(3).tolist()}")
assert all_features['PERMNO'].str.len().eq(5).all(), "❌ PERMNOs not all 5 digits!"
print("✓ All PERMNOs are 5-digit zero-padded strings")

# 3. FEATURE AVAILABILITY (after Feb 2024 when all lags populated)
print("\n3. FEATURE AVAILABILITY (Post Feb-2024)")
print("-" * 40)
mature_data = all_features[all_features['MthCalDt'] >= '2024-02-29']
print(f"Mature data: {len(mature_data):,} observations from {mature_data['MthCalDt'].min()}")

for i, feat in enumerate(feature_list, 1):
    if feat in mature_data.columns:
        pct_available = (mature_data[feat].notna().sum() / len(mature_data) * 100)
        status = "✓" if pct_available > 80 else "⚠️"
        print(f"   {status} {i:2d}. {feat:15s} - {pct_available:5.1f}% available")
    else:
        print(f"   ❌ {i:2d}. {feat:15s} - MISSING")

# 4. S&P 500 MEMBERSHIP CHECK
print("\n4. S&P 500 MEMBERSHIP")
print("-" * 40)
yearly_sp = all_features[all_features['in_sp500']==1].groupby(
    all_features['MthCalDt'].dt.year
)['PERMNO'].nunique()
print("Unique S&P 500 stocks by year:")
print(yearly_sp)
print(f"\nOverall S&P coverage: {all_features['in_sp500'].mean()*100:.1f}% of stock-months")
if yearly_sp.median() < 450 or yearly_sp.median() > 550:
    print("⚠️  Warning: S&P 500 count seems unusual (expected ~500)")

# 5. FUNDAMENTAL COVERAGE BY YEAR
print("\n5. FUNDAMENTAL COVERAGE BY YEAR")
print("-" * 40)
yearly_fund = all_features.groupby(
    all_features['MthCalDt'].dt.year
)[['bm', 'agr', 'operprof']].count()
print(yearly_fund)

# 6. OUTLIER DETECTION
print("\n6. OUTLIER DETECTION")
print("-" * 40)
for feat in ['bm', 'mom12m', 'retvol', 'agr']:
    if feat in mature_data.columns:
        print(f"\n{feat}:")
        print(mature_data[feat].describe(percentiles=[.01, .05, .95, .99]))

# 7. POINT-IN-TIME CHECK (fundamentals should lag market data)
print("\n7. POINT-IN-TIME VALIDATION")
print("-" * 40)
# Check that we don't have fundamentals available too early
# Sample a few stocks and verify fundamental dates respect the 4-month lag
sample_permnos = all_features['PERMNO'].unique()[:5]
for permno in sample_permnos:
    stock_data = all_features[all_features['PERMNO'] == permno].copy()
    # First non-null fundamental should be at least 4 months after data start
    first_market = stock_data['MthCalDt'].min()
    first_fundamental = stock_data[stock_data['bm'].notna()]['MthCalDt'].min()
    if pd.notna(first_fundamental):
        lag_months = (first_fundamental.year - first_market.year) * 12 + (first_fundamental.month - first_market.month)
        print(f"   PERMNO {permno}: Market starts {first_market.strftime('%Y-%m')}, "
              f"Fundamentals start {first_fundamental.strftime('%Y-%m')} (lag: {lag_months} months)")

# 8. FINAL SUMMARY
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"✓ Total observations: {len(all_features):,}")
print(f"✓ Date range: {all_features['MthCalDt'].min().strftime('%Y-%m')} to {all_features['MthCalDt'].max().strftime('%Y-%m')}")
print(f"✓ Unique stocks: {all_features['PERMNO'].nunique():,}")
print(f"✓ Mature data (post Feb-2024): {len(mature_data):,} observations")
print(f"✓ Features constructed: {len(feature_list)}")


DATA VALIDATION

1. DATE ALIGNMENT
----------------------------------------
Month-end day distribution:
MthCalDt
31    176815
30    101000
28     19042
29      6254
Name: count, dtype: int64
✓ All dates are proper month-ends

2. PERMNO FORMAT
----------------------------------------
Sample PERMNOs: ['10078', '10078', '10078']
✓ All PERMNOs are 5-digit zero-padded strings

3. FEATURE AVAILABILITY (Post Feb-2024)
----------------------------------------
Mature data: 7,287 observations from 2024-02-29 00:00:00
   ✓  1. mom1m           -  99.9% available
   ✓  2. mom6m           -  99.8% available
   ✓  3. mom12m          -  99.4% available
   ❌  4. mom36m          - MISSING
   ✓  5. chmom           -  99.4% available
   ✓  6. mvel1           - 100.0% available
   ✓  7. dolvol          - 100.0% available
   ✓  8. turn            - 100.0% available
   ✓  9. std_turn        -  99.7% available
   ✓ 10. retvol          -  99.7% available
   ✓ 11. maxret          - 100.0% available
   ✓ 12. ba

In [15]:
# ADD TARGET VARIABLE (next month's return)
print("\nCreating target variable...")
all_features = all_features.sort_values(['PERMNO', 'MthCalDt'])
all_features['target_ret'] = all_features.groupby('PERMNO')['MthRet'].shift(-1)

# Remove last observation for each stock (no target available)
all_features = all_features.dropna(subset=['target_ret'])

print(f"   Target variable created: {all_features['target_ret'].notna().sum():,} observations")
print(f"   Final dataset: {len(all_features):,} rows")


Creating target variable...
   Target variable created: 301,280 observations
   Final dataset: 301,280 rows


In [16]:
output_file = 'data.csv'
all_features.to_csv(output_file, index=False)

print(f"File: {output_file}")
print(f"Size: {len(all_features):,} observations")
print(f"Stocks: {all_features['PERMNO'].nunique():,}")
print(f"\nColumn list: {list(all_features.columns)}")

File: data.csv
Size: 301,280 observations
Stocks: 1,275

Column list: ['PERMNO', 'Ticker', 'MthCalDt', 'MthRet', 'mvel1', 'mom1m', 'mom6m', 'mom12m', 'chmom', 'retvol', 'dolvol', 'turn', 'std_turn', 'baspread', 'maxret', 'agr', 'operprof', 'bm', 'ep', 'rd_mve', 'in_sp500', 'target_ret']
